In [2]:
import pandas as pd

# ===== Scenario Inputs =====
scenarios = {
    "Conservative": {
        "adoption_rate": 0.03,      # 3% of students
        "trips_per_week": 2,        # observed trips per week
        "avg_ride_duration": 15,    # minutes
        "availability_pct": 0.80,   # 80% of operable minutes
        "buffer": 0.10,             # 10% extra for maintenance / spare
        "peak_fraction": 0.35,      # 45% of trips in peak window
        "peak_window_hr": 4,        # 3-hour peak window
        "docking_bays": 10
    },
    "Base": {
        "adoption_rate": 0.06,
        "trips_per_week": 3,
        "avg_ride_duration": 20,
        "availability_pct": 0.85,
        "buffer": 0.15,
        "peak_fraction": 0.45,
        "peak_window_hr": 4,
        "docking_bays": 12
    },
    "Optimistic": {
        "adoption_rate": 0.10,
        "trips_per_week": 4,
        "avg_ride_duration": 25,
        "availability_pct": 0.90,
        "buffer": 0.20,
        "peak_fraction": 0.55,
        "peak_window_hr": 4,
        "docking_bays": 15
    },
    "Commuter (Daily)": {
        "adoption_rate": 0.06,      # assume same 6% adoption, but heavier usage
        "trips_per_week": 10,       # daily commuting (5 days x 2 trips)
        "avg_ride_duration": 20,
        "availability_pct": 0.85,
        "buffer": 0.15,
        "peak_fraction": 0.60,      # higher share of trips concentrated in commute hours
        "peak_window_hr": 4,        # assume 4-hour combined AM + PM peak
        "docking_bays": 12
    }
}

student_population = 37700
operable_minutes_per_bike_total = 24 * 60  # 24 hours/day

# ===== Calculation =====
results = []

for name, params in scenarios.items():
    # Daily users
    daily_users = student_population * params["adoption_rate"]
    # Daily trips
    daily_trips = daily_users * (params["trips_per_week"] / 7)
    # Peak trips in peak window
    peak_trips = daily_trips * params["peak_fraction"]
    # Fleet needed during peak
    fleet_needed_peak = (peak_trips * params["avg_ride_duration"]) / (params["peak_window_hr"] * 60)
    fleet_needed_peak /= params["availability_pct"]  # adjust for availability
    fleet_needed_peak *= (1 + params["buffer"])      # adjust for buffer
    # Bikes per bay
    bikes_per_bay_peak = fleet_needed_peak / params["docking_bays"]
    
    results.append({
        "Scenario": name,
        "Daily Users": round(daily_users),
        "Daily Trips": round(daily_trips),
        "Peak Trips": round(peak_trips),
        "Fleet Needed (Peak)": round(fleet_needed_peak),
        "Bays": params["docking_bays"],
        "Bikes per Bay (Peak)": round(bikes_per_bay_peak, 1)
    })

# Convert to DataFrame for display
df_results = pd.DataFrame(results)
df_results


,Scenario,Daily Users,Daily Trips,Peak Trips,Fleet Needed (Peak),Bays,Bikes per Bay (Peak)
0,Conservative,1131,323,113,10,10,1.0
1,Base,2262,969,436,49,12,4.1
2,Optimistic,3770,2154,1185,165,15,11.0
3,Commuter (Daily),2262,3231,1939,219,12,18.2
